# 0. GPU check

* 이 코드는 Nvidia GPU를 사용하는 컴퓨터에서, train / test 데이터가 분리되어있는 csv 파일을 사용하는 것을 전제로 작성됨

In [1]:
import torch

if torch.cuda.is_available():
    device_count = torch.cuda.device_count()
    print("device_count: {}".format(device_count))
    for device_num in range(device_count):
        print("device {} capability {}".format(
            device_num,
            torch.cuda.get_device_capability(device_num)))
        print("device {} name {}".format(
            device_num, 
            torch.cuda.get_device_name(device_num)))
else:
    print("no cuda device")

device_count: 1
device 0 capability (8, 6)
device 0 name NVIDIA GeForce RTX 3080


In [2]:
if torch.cuda.is_available() :
    device = torch.device("cuda:0")
else : 
    device = torch.device("cpu")

In [3]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()
    
print_gpu_utilization()

GPU memory occupied: 428 MB.


* 모델 훈련과정에서 GPU 메모리 용량 초과 시, 개발서버 콘솔에서 직접 `nvidia-smi` 명령어 실행 후 메모리를 점유하고 있는 process의 PID를 찾아 `sudo kill -9 {pid}` 로 프로세스 종료해주면 됨

# 1. Import packages

In [4]:
## Need to check if packages are compatible
# !pip install accelerate nvidia-ml-py3
# !pip install datasets==2.4.0
# !pip install huggingface_hub==0.9.1
# !pip install transformers==4.22.1 # bf16, tf32 등 사용하려면 4.2 이상 필요
# !pip install pyarrow==9.0.0

* huggingface_hub와 transformers 간 호환가능한 버전 확인 필요
* 만약 성능 테스트를 위해 datasets api를 사용할거라면 datasets 역시 호환 가능 버전 확인해야 함
* 세 가지 dependencies를 사용한다는 가정 하에, pyarrow 라이브러리도 필요.

In [5]:
import transformers
import datasets
import huggingface_hub
import pyarrow

print(transformers.__version__)
print(datasets.__version__)
print(huggingface_hub.__version__)
print(pyarrow.__version__)

# 4.22.1
# 2.4.0
# 0.9.1
# 9.0.0

4.22.1
2.4.0
0.9.1
9.0.0


In [6]:
import os
import re
import math
import numpy as np
import pandas as pd

# 'You can use tf32' if you are acessing Ampere hardware
import torch
torch.backends.cuda.matmul.allow_tf32 = True

from datasets import load_dataset, load_metric, ClassLabel
from sklearn.utils.class_weight import compute_class_weight
import ray
from ray import tune
from ray.tune import CLIReporter
from ray.tune.examples.pbt_transformers.utils import (
    download_data,
    build_compute_metrics_fn,
)
from ray.tune.schedulers import PopulationBasedTraining
from transformers import (
    glue_tasks_num_labels,
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    GlueDataset,
    GlueDataTrainingArguments,
    TrainingArguments,
)

In [29]:
from transformers import EarlyStoppingCallback

# 2. Import Data

* xxx_train.csv, xxx_test.csv 파일은 아래 형식으로 전처리된 csv 파일이어야 함 (column name: `text`, `label`)


<table class="features-table">
  <tr>
    <th class="mdc-text-light-green-600", style="text-align:center">
    text
    </th>
    <th class="mdc-text-purple-600", style="text-align:center">
    label
    </th>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
    </td>
    <td class="mdc-bg-purple-50">
      0
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      Ok lar... Joking wif u oni...
    </td>
    <td class="mdc-bg-purple-50">
      0
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)
    </td>
    <td class="mdc-bg-purple-50">
      1
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      U dun say so early hor... U c already then say...
    </td>
    <td class="mdc-bg-purple-50">
      0
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      Nah I don't think he goes to usf, he lives around here though
    </td>
    <td class="mdc-bg-purple-50">
      0
    </td>
  </tr>
</table>

In [8]:
data_name = "IMDB" ## covid_articles / financial_news / IMDB / naver_movie_review / spam

dataset = load_dataset('csv', data_files={'train': f'../data_splited/{data_name}_train.csv',
                                          'test': f'../data_splited/{data_name}_test.csv'})
dataset

Using custom data configuration default-5e9b2acfce9f0b59
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-5e9b2acfce9f0b59/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 39999
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 9999
    })
})

# 3. Data Preprocessing

* load_dataset 함수로 불러온 데이터를 수정할 때는 수정 내용을 담은 함수를 만들고, 이를 map 함수로 각 원소에 적용함 ([링크](https://huggingface.co/docs/datasets/v1.4.0/processing.html#processing-data-row-by-row)에서 확인)

In [9]:
## remove specal characters

def remove_sp(example):
    example["text"]=re.sub(r'[^a-z|A-Z|0-9|ㄱ-ㅎ|ㅏ-ㅣ|가-힣| ]+', '', str(example["text"]))
    return example

dataset = dataset.map(remove_sp)

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-5e9b2acfce9f0b59/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-0c659aeae188f731.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-5e9b2acfce9f0b59/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-170d6a353647063a.arrow


In [10]:
## label encoding

labels = list(set(dataset["train"]["label"] + dataset["test"]["label"]))
num_labels = len(labels)

def encoding_label(example):
    str_to_int = ClassLabel(num_classes=num_labels, names=labels)
    example["label"]=str_to_int.str2int(example["label"])
    return example

if type(labels[0]) == str:
    dataset = dataset.map(encoding_label)
    
print(num_labels)

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-5e9b2acfce9f0b59/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-82b60351b1fd1890.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-5e9b2acfce9f0b59/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-0fbc2fb5dafc34c1.arrow


2


# 4. Load PLM & Tokenizing

In [11]:
model_name = "bert-base-cased"
# model_name = "bert-base-multilingual-cased"
# model_name = "xlm-roberta-base"

# model_name = "klue/bert-base"
# model_name = "klue/roberta-base"


In [12]:
# Download cache tokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [13]:
def tokenize_function(examples):
    tokenized_batch = tokenizer(examples["text"], padding="max_length", truncation=True) # padding : ['longest', 'max_length', 'do_not_pad']
    return tokenized_batch

In [14]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-5e9b2acfce9f0b59/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-1b3db1ac7e2ce261.arrow


  0%|          | 0/10 [00:00<?, ?ba/s]

In [15]:
# train_dataset = tokenized_datasets["train"].shuffle(seed=1919).select(range(0,math.floor(len(tokenized_datasets["train"])*0.7)))
# eval_dataset = tokenized_datasets["train"].shuffle(seed=1919).select(range(math.floor(len(tokenized_datasets["train"])*0.7), len(tokenized_datasets["train"])))
# test_dataset = tokenized_datasets["test"]

In [16]:
# data for test
train_dataset = tokenized_datasets["train"].shuffle(seed=1919).select(range(1000))
eval_dataset = tokenized_datasets["train"].shuffle(seed=1919).select(range(1000))
test_dataset = tokenized_datasets["test"]

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/csv/default-5e9b2acfce9f0b59/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-e9e4b6fdb429ddad.arrow
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/csv/default-5e9b2acfce9f0b59/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-e9e4b6fdb429ddad.arrow


# 5. Check class weights

In [17]:
def class_weight(train_dataset) :
    
    train_labels = np.array(train_dataset["label"])
    class_weights = compute_class_weight(class_weight = 'balanced', classes = np.unique(train_labels), y = train_labels)
    
    weights = torch.tensor(class_weights, dtype = torch.float)
    
    return weights

In [18]:
weights = class_weight(train_dataset)
print(weights)

tensor([1.0225, 0.9785])


# 6. Modeling

In [24]:
## Customize training strategy

task_data_dir = "test-model"
gpus_per_trial = 1
cpus_per_trial = 16
n_trials = 5
metric = load_metric("f1") # atasets.list_metrics() 
seed = 818

In [25]:
# Download model and features

config = AutoConfig.from_pretrained(
    model_name, 
    num_labels=num_labels
)

def model_init():
    return AutoModelForSequenceClassification.from_pretrained(
        model_name,
        config=config
        )

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/a8d257ba9925ef39f3036bfc338acf5283c512d9/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.22.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}



In [26]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions=np.argmax(logits, axis = -1)
    return metric.compute(predictions=predictions, references=labels)

```python
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=1,   # batch size per device during training
    per_device_eval_batch_size=10,   # batch size for evaluation
    warmup_steps=1000,               # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=200,               # How often to print logs
    do_train=True,                   # Perform training
    do_eval=True,                    # Perform evaluation
    evaluation_strategy="epoch",     # evalute after each epoch
    gradient_accumulation_steps=64,  # total number of steps before back propagation
    fp16=True,                       # Use mixed precision
    fp16_opt_level="02",             # mixed precision mode
    run_name="ProBert-BFD-MS",       # experiment name
    seed=3                           # Seed for experiment reproducibility 3x3
)
```

In [30]:
training_args = TrainingArguments(
    output_dir=".",
    learning_rate=2e-5, # config
    do_train=True,
    do_eval=True,
    no_cuda=gpus_per_trial <= 0,
    evaluation_strategy="steps",
    save_strategy="steps",
    metric_for_best_model="accuracy",
    greater_is_better=True,
    load_best_model_at_end=True,
    num_train_epochs=2,  # config
    max_steps=-1,  # config
    per_device_train_batch_size=8,  # config
    per_device_eval_batch_size=8,  # config
    warmup_steps=0,
    warmup_ratio=0.1,  # config
    weight_decay=0.1,  # config
    logging_dir="./logs",
    skip_memory_metrics=True,
    report_to="none",
    fp16=True,
    # bf16=True,
    # tf32=True,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    seed=seed,
    eval_steps = 50
    )
    
# trainer = Trainer(
#     model_init=model_init,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=eval_dataset,
#     compute_metrics=compute_metrics,
#     )

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss
        weight = weights.to(device)
        loss_fct = torch.nn.CrossEntropyLoss(weight=weight)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss
    
trainer = CustomTrainer(
    model_init=model_init,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
    )

PyTorch: setting up devices
loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/a8d257ba9925ef39f3036bfc338acf5283c512d9/pytorch_model.bin
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly i

In [ ]:
# Hyperparameter tuning with ray tune

tune_config = {
#     "per_device_train_batch_size": tune.choice([2, 4, 8]),
    "num_train_epochs": tune.choice([2, 5, 10]),
#     "num_train_epochs": [x for x in range(2, 21)],
}

# PopulationBasedTraining
# worker might copy the model parameters from a better performing worker or explore new hyperparameters by changing the current values randomly
# cf. ASHAScheduler
scheduler = PopulationBasedTraining(
    time_attr="training_iteration",
    metric="eval_f1",
    mode="max",
    perturbation_interval=1,
    hyperparam_mutations={
#         "num_train_epochs": [x for x in range(2, 21)],
        "weight_decay": tune.uniform(0.0, 0.3), # tune.uniform(1, 10) == np.random.uniform(1, 10)
        "learning_rate": tune.uniform(1e-5, 5e-5),
        "warmup_ratio": tune.uniform(0.0, 0.3),
#         # Perturb factor3 by changing it to an adjacent value, e.g.
#         # 10 -> 1 or 10 -> 100. Resampling will choose at random.
#         "factor_3": [1, 10, 100, 1000, 10000],
#         # Using tune.choice is NOT equivalent to the above.
#         # factor_4 is treated as a continuous hyperparameter.
#         "factor_4": tune.choice([1, 10, 100, 1000, 10000]),
    },
)


reporter = CLIReporter(
    parameter_columns={
        "weight_decay": "w_decay",
        "learning_rate": "lr",
        "per_device_train_batch_size": "train_bs/gpu",
        "num_train_epochs": "num_epochs",
    },
    metric_columns=["eval_f1", "eval_loss", "epoch", "training_iteration"],
)

result = trainer.hyperparameter_search(
    direction = "maximize",
    hp_space = lambda _: tune_config,
    backend="ray",
    n_trials=n_trials,
    resources_per_trial={"cpu": cpus_per_trial, "gpu": gpus_per_trial},
    scheduler=scheduler,
    keep_checkpoints_num=1,
    checkpoint_score_attr="training_iteration",
    stop=None,
    progress_reporter=reporter,
    local_dir="./test-results",
    name="tune_transformer_pbt",
    log_to_file=True,
)

(pid=2901398) 2022-10-14 09:15:39.436636: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


== Status ==
Current time: 2022-10-14 09:15:37 (running for 00:00:00.28)
Memory usage on this node: 11.4/31.1 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 16.0/20 CPUs, 1.0/1 GPUs, 0.0/15.25 GiB heap, 0.0/7.62 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /workspace/syc/BERT_classification_binary/test-results/tune_transformer_pbt
Number of trials: 5/5 (4 PENDING, 1 RUNNING)
+------------------------+----------+--------------------+-----------+-------------+----------------+--------------+
| Trial name             | status   | loc                |   w_decay |          lr | train_bs/gpu   |   num_epochs |
|------------------------+----------+--------------------+-----------+-------------+----------------+--------------|
| _objective_c4104_00000 | RUNNING  | 172.17.0.3:2901398 | 0.0885716 | 4.40175e-05 |                |            2 |
| _objective_c4104_00001 | PENDING  |                    | 0.215004  | 1.48101e-05 |                |            5

(_objective pid=2901398) Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
(_objective pid=2901398) - This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(_objective pid=2901398) - This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(_objective pid=290139

== Status ==
Current time: 2022-10-14 09:15:45 (running for 00:00:07.93)
Memory usage on this node: 15.7/31.1 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 16.0/20 CPUs, 1.0/1 GPUs, 0.0/15.25 GiB heap, 0.0/7.62 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /workspace/syc/BERT_classification_binary/test-results/tune_transformer_pbt
Number of trials: 5/5 (4 PENDING, 1 RUNNING)
+------------------------+----------+--------------------+-----------+-------------+----------------+--------------+
| Trial name             | status   | loc                |   w_decay |          lr | train_bs/gpu   |   num_epochs |
|------------------------+----------+--------------------+-----------+-------------+----------------+--------------|
| _objective_c4104_00000 | RUNNING  | 172.17.0.3:2901398 | 0.0885716 | 4.40175e-05 |                |            2 |
| _objective_c4104_00001 | PENDING  |                    | 0.215004  | 1.48101e-05 |                |            5

 15%|█▍        | 9/62 [00:05<00:33,  1.56it/s]


...

 32%|███▏      | 50/155 [00:32<01:07,  1.55it/s]
(_objective pid=2901654) 
  0%|          | 0/125 [00:00<?, ?it/s]
(_objective pid=2901654) 
  3%|▎         | 4/125 [00:00<00:03, 32.16it/s]
(_objective pid=2901654) 
  6%|▋         | 8/125 [00:00<00:04, 26.84it/s]
(_objective pid=2901654) 
  9%|▉         | 11/125 [00:00<00:04, 25.72it/s]
(_objective pid=2901654) 
 11%|█         | 14/125 [00:00<00:04, 25.10it/s]
(_objective pid=2901654) 
 14%|█▎        | 17/125 [00:00<00:04, 24.75it/s]
(_objective pid=2901654) 
 16%|█▌        | 20/125 [00:00<00:04, 24.52it/s]
(_objective pid=2901654) 
 18%|█▊        | 23/125 [00:00<00:04, 24.39it/s]
(_objective pid=2901654) 
 21%|██        | 26/125 [00:01<00:04, 24.31it/s]
(_objective pid=2901654) 
 23%|██▎       | 29/125 [00:01<00:03, 24.24it/s]
(_objective pid=2901654) 
 26%|██▌       | 32/125 [00:01<00:03, 24.21it/s]
(_objective pid=2901654) 
 28%|██▊       | 35/125 [00:01<00:03, 24.17it/s]


== Status ==
Current time: 2022-10-14 09:17:02 (running for 00:01:25.47)
Memory usage on this node: 15.7/31.1 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 16.0/20 CPUs, 1.0/1 GPUs, 0.0/15.25 GiB heap, 0.0/7.62 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /workspace/syc/BERT_classification_binary/test-results/tune_transformer_pbt
Number of trials: 5/5 (1 PAUSED, 3 PENDING, 1 RUNNING)
+------------------------+----------+--------------------+-----------+-------------+----------------+--------------+-----------+-------------+---------+----------------------+
| Trial name             | status   | loc                |   w_decay |          lr | train_bs/gpu   |   num_epochs |   eval_f1 |   eval_loss |   epoch |   training_iteration |
|------------------------+----------+--------------------+-----------+-------------+----------------+--------------+-----------+-------------+---------+----------------------|
| _objective_c4104_00001 | RUNNING  | 172.17

(_objective pid=2901654) 
 30%|███       | 38/125 [00:01<00:03, 24.16it/s]
(_objective pid=2901654) 
 33%|███▎      | 41/125 [00:01<00:03, 24.12it/s]
(_objective pid=2901654) 
 35%|███▌      | 44/125 [00:01<00:03, 24.12it/s]
(_objective pid=2901654) 
 38%|███▊      | 47/125 [00:01<00:03, 24.10it/s]
(_objective pid=2901654) 
 40%|████      | 50/125 [00:02<00:03, 24.11it/s]
(_objective pid=2901654) 
 42%|████▏     | 53/125 [00:02<00:02, 24.11it/s]
(_objective pid=2901654) 
 45%|████▍     | 56/125 [00:02<00:02, 24.10it/s]
(_objective pid=2901654) 
 47%|████▋     | 59/125 [00:02<00:02, 24.06it/s]
(_objective pid=2901654) 
 50%|████▉     | 62/125 [00:02<00:02, 24.06it/s]
(_objective pid=2901654) 
 52%|█████▏    | 65/125 [00:02<00:02, 24.07it/s]
(_objective pid=2901654) 
 54%|█████▍    | 68/125 [00:02<00:02, 24.07it/s]
(_objective pid=2901654) 
 57%|█████▋    | 71/125 [00:02<00:02, 24.06it/s]
(_objective pid=2901654) 
 59%|█████▉    | 74/125 [00:03<00:02, 24.07it/s]
(_objective pid=2901654) 

Result for _objective_c4104_00001:
  date: 2022-10-14_09-17-06
  done: false
  epoch: 1.61
  eval_f1: 0.7258771929824562
  eval_loss: 0.5450262427330017
  eval_runtime: 5.2179
  eval_samples_per_second: 191.649
  eval_steps_per_second: 23.956
  experiment_id: a6f8c91886154a9681070e0400ff5285
  hostname: 3481a8a2ae33
  iterations_since_restore: 1
  node_ip: 172.17.0.3
  objective: 0.7258771929824562
  pid: 2901654
  time_since_restore: 41.554603576660156
  time_this_iter_s: 41.554603576660156
  time_total_s: 41.554603576660156
  timestamp: 1665739026
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: c4104_00001
  warmup_time: 0.0019211769104003906
  
(_objective pid=2901654) {'eval_loss': 0.5450262427330017, 'eval_f1': 0.7258771929824562, 'eval_runtime': 5.2179, 'eval_samples_per_second': 191.649, 'eval_steps_per_second': 23.956, 'epoch': 1.61}


(_objective pid=2901654) 
                                                
100%|██████████| 125/125 [00:05<00:00, 23.42it/s]early stopping required metric_for_best_model, but did not find eval_accuracy so early stopping is disabled
(_objective pid=2901654) 
                                                 
 32%|███▏      | 50/155 [00:37<01:19,  1.32it/s]


In [ ]:
result

In [ ]:
for n, v in result.hyperparameters.items():
    setattr(trainer.args, n, v)

In [ ]:
# trainer.args

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.predict(test_dataset=test_dataset)

(_objective pid=2900563) 
                                                
100%|██████████| 125/125 [00:05<00:00, 24.21it/s]early stopping required metric_for_best_model, but did not find eval_accuracy so early stopping is disabled
(_objective pid=2900563) 
                                                 


(_objective pid=2900563) {'eval_loss': 0.6048755049705505, 'eval_f1': 0.6558891454965358, 'eval_runtime': 5.1791, 'eval_samples_per_second': 193.083, 'eval_steps_per_second': 24.135, 'epoch': 1.61}


In [ ]:
# model_path = "test-model"
# trainer.model.save_pretrained(model_path)
# tokenizer.save_pretrained(model_path)

# Reference

https://bo-10000.tistory.com/154  
https://huggingface.co/blog/ray-tune  
https://docs.ray.io/en/latest/tune/examples/pbt_transformers.html  
https://wood-b.github.io/post/a-novices-guide-to-hyperparameter-optimization-at-scale/#schedulers-vs-search-algorithms  
https://docs.ray.io/en/latest/tune/api_docs/search_space.html  
https://docs.ray.io/en/latest/tune/tutorials/tune-advanced-tutorial.html  
https://docs.ray.io/en/latest/tune/api_docs/schedulers.html  
https://blog.ml.cmu.edu/2018/12/12/massively-parallel-hyperparameter-optimization/  
https://docs.ray.io/en/latest/tune/faq.html  
https://docs.ray.io/en/latest/tune/api_docs/schedulers.html#population-based-training-tune-schedulers-populationbasedtraining  
https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.hyperparameter_search  
https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#optuna-tune-search-optuna-optunasearch  